## Company Information:
The data set is based upon <a src="https://www.kaggle.com/prateikmahendra/loan-data"> Lending Club Information </a>. 
- TheIrish Dummy Banks  is a  peer to peer lending bank based in the ireland, in which bank provide funds for potential borrowers and bank earn a profit depending on the risk they take (the borrowers credit score). Irish Fake bank provides loan to their loyal customers. The complete data set is borrowed from Lending Club For more basic information about the company please check out the wikipedia article about the company. This dataset is copied and clean from kaggle but it has been changed. The any kind of similarity is just for learning purposes. I dont have any intention for palgrisim I just like to be clear myself. <br><br>


<a src="https://en.wikipedia.org/wiki/Lending_Club"> Lending Club Information </a>


In [1]:
# Import our libraries we are going to use for our data analysis.
import tensorflow as tf
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Plotly visualizations
from plotly import tools
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


# For oversampling Library (Dealing with Imbalanced Datasets)
from imblearn.over_sampling import SMOTE
from collections import Counter
from IPython.display import HTML
import warnings; warnings.simplefilter('ignore')


% matplotlib inline

df = pd.read_csv("C:/Users/mrferozi/Documents/Panada_ml/loan/lending-club-loan-data/loan_final313.csv",low_memory=False, index_col=0)

# Copy of the dataframe
original_df = df.copy()

import warnings; warnings.simplefilter('ignore')

C:\Users\mrferozi\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df.dtypes

year                      int64
issue_d                  object
final_d                   int64
emp_length_int          float64
home_ownership           object
home_ownership_cat        int64
income_category          object
annual_inc                int64
income_cat                int64
loan_amount               int64
term                     object
term_cat                  int64
application_type         object
application_type_cat      int64
purpose                  object
purpose_cat               int64
interest_payments        object
interest_payment_cat      int64
loan_condition           object
loan_condition_cat        int64
interest_rate           float64
grade                    object
grade_cat                 int64
dti                     float64
total_pymnt             float64
total_rec_prncp         float64
recoveries              float64
installment             float64
region                   object
dtype: object

## Feature selection

How do we choose which features to include in the model? We're going to use **train/test split** (and eventually **cross-validation**).

Why not use of **p-values** or **R-squared** for feature selection?

- Linear models rely upon **a lot of assumptions** (such as the features being independent), and if those assumptions are violated, p-values and R-squared are less reliable. Train/test split relies on fewer assumptions.
- Features that are unrelated to the response can still have **significant p-values**.
- Adding features to your model that are unrelated to the response will always **increase the R-squared value**, and adjusted R-squared does not sufficiently account for this.
- p-values and R-squared are **proxies** for our goal of generalization, whereas train/test split and cross-validation attempt to **directly estimate** how well the model will generalize to out-of-sample data.



## Comparing models with train/test split and RMSE

In [3]:
# create a list of features
feature_cols = ['emp_length_int', 'annual_inc','loan_amount',
                'interest_rate','dti','home_ownership_cat',
               'income_cat','total_pymnt','purpose_cat','grade_cat',
               'application_type_cat','term_cat','year']

In [8]:
from sklearn.cross_validation import train_test_split

# define a function that accepts a list of features and returns testing RMSE
def train_test_rmse(feature_cols):
    X = df[feature_cols]
    y = df.loan_condition_cat
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    y_pred = linreg.predict(X_test)
    return np.sqrt(metrics.mean_squared_error(y_test, y_pred))

In [14]:
# compare different sets of features
print (train_test_rmse(['emp_length_int','annual_inc','loan_amount','interest_rate','dti','home_ownership_cat']))


0.2602041769988056


In [15]:
print (train_test_rmse(['emp_length_int','annual_inc','loan_amount','interest_rate','dti','home_ownership_cat','income_cat','total_pymnt','purpose_cat','grade_cat','application_type_cat','term_cat','year']))

0.2508278160811187


In [16]:
print (train_test_rmse(['emp_length_int','annual_inc','loan_amount','interest_rate','dti','home_ownership_cat','income_cat','total_pymnt','purpose_cat','grade_cat','application_type_cat','term_cat']))

0.2583471389714144


In [18]:
print (train_test_rmse(['emp_length_int','annual_inc','loan_amount','interest_rate','dti','home_ownership_cat','income_cat','total_pymnt','purpose_cat','grade_cat','application_type_cat','term_cat','year']))

0.2508278160811187


In [20]:
print (train_test_rmse(['emp_length_int','annual_inc','loan_amount','interest_rate','dti','home_ownership_cat','income_cat','total_pymnt','purpose_cat','term_cat','year']))

0.25127812204823347


In [21]:
print (train_test_rmse(['emp_length_int','annual_inc','loan_amount','interest_rate','dti','home_ownership_cat','income_cat','total_pymnt','purpose_cat','grade_cat','application_type_cat','term_cat','year']))

0.2508278160811187


### Classification Accuracy (Logistic Regression)

- That is mean that all features have equal importance

In [25]:
# create a list of features
feature_cols = ['emp_length_int', 'annual_inc','loan_amount',
                'interest_rate','dti','home_ownership_cat',
               'income_cat','total_pymnt','purpose_cat','grade_cat',
               'application_type_cat','term_cat','year']

In [26]:
X = df[feature_cols]
y = df.loan_condition_cat

In [27]:
## Modeling process
# spilt X and y into training and testing sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [28]:
# train a logistic regression model on the training set
from sklearn.linear_model import LogisticRegression
Logreg = LogisticRegression()
Logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [29]:
# make class prediction for the testing set
y_pred_class = Logreg.predict(X_test)

- Classification Accuracy : percentage of correct prediction

In [31]:
# calculate Accuracy
from sklearn import metrics
print((metrics.accuracy_score(y_test, y_pred_class))*100)

92.33518898329916


- Null accuracy : accuracy that could be achieved by always predicting the most frequent class

In [32]:
# examine the class distribution of the testing set (using panda series method)
y_test.value_counts()

0    204841
1     17004
Name: loan_condition_cat, dtype: int64

- System Predict Good loan = 204841 and Bad Loan =  17004

In [33]:
y_test.mean()

0.0766481101670085

In [35]:
# calculate the percentage of zeros
print ((1- y_test.mean())*100)

92.33518898329916


In [36]:
# claculate null accuracy ( for binary classification problem coded as 0/1)
print (max(y_test.mean(), 1- y_test.mean()))

0.9233518898329915


- Comparing the true and predicted response values

In [44]:
# print the first 25 true and predicted responses
from __future__ import print_function
print('True:', y_test.values[0:25])
print('Pred:', y_pred_class[0:25])

True: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Pred: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


**Conclusion:**

- Classification accuracy is the **easiest classification metric to understand**
- But, it does not tell you the **underlying distribution** of response values
- And, it does not tell you what **"types" of errors** your classifier is making